In [1]:
import polars as pl
import pandas as pd
import numpy as np
import math as math

In [3]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

In [11]:
# normalize data
players = players.with_columns([pl.col('nflId').cast(str)])
plays = plays.join(games,on='gameId')
plays = plays.with_columns([
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId')
])

tracking = tracking.with_columns(
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId'),
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str) + '-'
     + pl.col('nflId').cast(str)).alias('uniquePlayerId'),
)

# normalize position
tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(53.3-pl.col('y')).otherwise(pl.col('y')).alias('adjustedX'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('x')).otherwise(120-pl.col('x')).alias('adjustedY')
])

tracking=tracking.with_columns([
    pl.when(pl.col('event')=='ball_snap').then(pl.col('frameId')).otherwise(-1).alias('startingFrameId'),
])
tracking=tracking.with_columns([
    pl.col('startingFrameId').max().over(pl.col('uniquePlayId')),
])
tracking=tracking.with_columns([
    (pl.col('frameId') - pl.col('startingFrameId')).alias('framesSinceSnap'),
])

# normalize orientation 'o' and direction 'dir'
# convert 'NA' to 0
replacement_values = {'NA': '0'}
tracking = tracking.with_columns(
    pl.col('o').apply(lambda x: replacement_values.get(x, x)),
    pl.col('dir').apply(lambda x: replacement_values.get(x, x)),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(pl.col('dir').cast(pl.Float64)).otherwise(180-pl.col('dir').cast(pl.Float64)).alias('adjustedDir'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('o').cast(pl.Float64)).otherwise(180-pl.col('o').cast(pl.Float64)).alias('adjustedO'),
])

In [39]:
def is_in_vision_cone(index, df):
    # Calculate angle between origin and point
    angle = math.atan2(y1 - y_origin, x1 - x_origin)
    # Convert angle to degrees
    angle_degrees = math.degrees(angle) % 360
    # Calculate angle difference between the point and the cone's direction
    angle_difference = abs((angle_degrees - degree + 360) % 360 - 180)
    
    # Check if the point is within the cone's vision
    if angle_difference <= 0.5 * degree and math.sqrt((x1 - x_origin) ** 2 + (y1 - y_origin) ** 2) <= radius:
        return True
    else:
        return False

In [91]:
cone_angle = 25 # degrees
max_distance = 10 # feet

def is_in_vision_cone(player1_row: pl.DataFrame, player2_row) -> bool:
    return (is_in_angle(player1_row, player2_row) and is_in_distance(player1_row, player2_row))

def is_in_angle(player1, player2) -> bool:
    half_cone_angle = cone_angle / 2;

    y_dist = player2.select(pl.col('adjustedY')).item() - player1.select(pl.col('adjustedY')).item()
    x_dist = player2.select(pl.col('adjustedX')).item() - player1.select(pl.col('adjustedX')).item()

    player1_orientation = player1.select(pl.col('adjustedO')).item()

    angle = math.degrees(math.atan2(y_dist, x_dist))

    if player1_orientation - half_cone_angle <= angle <= player1_orientation + half_cone_angle:
        return True
    
    return False

def is_in_distance(player1_index, player2_index) -> bool:
    y_dist = abs(player1.select(pl.col('adjustedY')).item() - player2.select(pl.col('adjustedY')).item())
    x_dist = abs(player1.select(pl.col('adjustedX')).item() - player2.select(pl.col('adjustedX')).item())

    distance_between_players = math.sqrt(x_dist**2 + y_dist**2)
    
    if distance_between_players <= max_distance:
        return True
    
    return False

In [92]:
gameId = 2022090800
playId = 56
frameId = 1

current_game_play_frame = (
    tracking.filter(pl.col('gameId')==gameId)
        .filter(pl.col('playId')==playId)
        .filter(pl.col('frameId')==frameId)
)

# validating :)
data = {
    'adjustedX': [50.00, 50.00],
    'adjustedY': [45.00, 46.00],
    'adjustedO': [90.00, 270.00]
}

test_df = pl.DataFrame(data)

player1 = test_df[0]
player2 = test_df[1]

print(player1)
print(player2)

is_in_vision_cone(player1, player2)

shape: (1, 3)
┌───────────┬───────────┬───────────┐
│ adjustedX ┆ adjustedY ┆ adjustedO │
│ ---       ┆ ---       ┆ ---       │
│ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╡
│ 50.0      ┆ 45.0      ┆ 90.0      │
└───────────┴───────────┴───────────┘
shape: (1, 3)
┌───────────┬───────────┬───────────┐
│ adjustedX ┆ adjustedY ┆ adjustedO │
│ ---       ┆ ---       ┆ ---       │
│ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╡
│ 50.0      ┆ 46.0      ┆ 270.0     │
└───────────┴───────────┴───────────┘


True

In [87]:
type(tracking)

polars.dataframe.frame.DataFrame